In [341]:
import numpy as np

# -------------------------------------------------------------------------------------------------------------#
# sudoku solving program

# check the state is goal
def is_goal(state):
    
    # check all positions are filled with numbers
    for row in range(9):
        for col in range(9):
            if state[row][col] == 0:
                return False
    
    # check all rows and columns to have exactly one numbers from 1~9
    for row in range(9):
        row_sum = 0
        col_sum = 0
        for col in range(9):
            row_sum += state[row][col]
            col_sum += state[col][row]
        if row_sum != 45 or col_sum != 45:
            return False
         
    # check all boxes to have exactly one numbers from 1~9
    for i in range(3):
        for j in range(3):
            box_sum = 0
            for m in range(3):
                for n in range(3):
                    box_sum += state[i*3+m][j*3+n]
            if box_sum != 45:
                return False
   
    # if all conditions are met, the state is goal state
    return True
           
        
# find possible numbers that can go into given position (state[row][col])
def possible_numbers(state, row, col):
    
    pos_nums = [1,2,3,4,5,6,7,8,9]
    
    # check the existing numbers for row and column
    for i in range(9):
        if state[row][i] in pos_nums:
            pos_nums.remove(state[row][i])
        if state[i][col] in pos_nums:
            pos_nums.remove(state[i][col])
        
    # check the existing numbers for box
    row //= 3
    col //= 3
    for m in range(row*3, (row+1)*3):
        for n in range(col*3, (col+1)*3):
            if state[m][n] in pos_nums:
                pos_nums.remove(state[m][n])
                
    return pos_nums


# find empty positions in state
def empty_positions(state):
    empty_pos = []
    for row in range(9):
        for col in range(9):
            if state[row][col] == 0:
                empty_pos.append((row, col))
    return empty_pos
    

# solving by dfs of given state
def solve_dfs(state):
    
    # check whether the state is goal
    if is_goal(state):
        global cnt
        cnt += 1
        #print('solved')
        # print the goal state
        #for row in state:
        #    print(*row)
    
    else:
        # list of positions needs to be filled given a state
        to_fill = empty_positions(state)
        
        # By row and column, choose a position which will be filled
        (row, col) = to_fill[0]

        # find the list of possible numbers that can be assigned to given row and column
        pos_nums = possible_numbers(state, row, col)

        for num in pos_nums:
            # assign a possible number to empty position 
            state[row][col] = num
            
            # continue to move on to the next state
            solve_dfs(state)
            
            # if it cannot be solved, initialize the filled position
            state[row][col] = 0

            
# -------------------------------------------------------------------------------------------------------------#
# sudoku problem generating program

blank = np.zeros(9).astype(int)

def possible_nums(state):
    
    pos_nums = [1,2,3,4,5,6,7,8,9]
    
    for i in range(9):
        if state[i] in pos_nums:
            pos_nums.remove(state[i])
                
    return pos_nums

for i in range(9):
    pos_nums = possible_nums(blank)
    while True:
        cand_num = np.random.randint(1,10)
        if cand_num in pos_nums:
            blank[i] = cand_num
            break

r1 = blank
r2 = np.roll(r1, 3)
r3 = np.roll(r2, 3)
r4 = np.roll(r1, 1)
r5 = np.roll(r4, 3)
r6 = np.roll(r5, 3)
r7 = np.roll(r1, 2)
r8 = np.roll(r7, 3)
r9 = np.roll(r8, 3)

# randomly generate complete sudoku
start = np.concatenate([r1,r2,r3,r4,r5,r6,r7,r8,r9], axis=0)
boxed = start.reshape(9,9)
# shuffle rows and columns
rep = boxed.copy()
rep[0], rep[1] = boxed[1], boxed[0]
rep2 = rep.copy()
rep2[:,3], rep2[:,4] = rep[:,4], rep[:,3]
rep3 = rep2.copy()
rep3[7], rep3[8] = rep2[8], rep2[7]
rep4 = rep3.copy()
rep4[:,6], rep4[:,7] = rep3[:,7], rep3[:,6]
start = rep4.flatten()

i = 0
while True:
    test = start.copy()
    # increase the number of erased elements by 1
    i += 1
    # randomly erase elements in given sudoku
    for j in range(i):
        a = np.random.randint(0,81)
        if test[a] != 0:
            test[a] = 0
        elif test[a] == 0:
            a = np.random.randint(0,81)
            test[a] = 0
    
    boxed = test.reshape((9,9))
    prob_state = boxed.tolist()
    
    # count the number of solution for generated sudoku
    global cnt
    cnt = 0
    solve_dfs(prob_state)
    
    # if the sudoku has more than 1 solution break the while loop and print the problem
    if cnt >= 2:
        if (i >= 1) and (i < 20):
            print('Easy Problem, ' + str(i) + ' grids to fill')
            for row in prob_state:
                print(*row)
        elif (i >= 20) and (i < 45):
            print('Medium Problem, ' + str(i) + ' grids to fill')
            for row in prob_state:
                print(*row)
        elif (i >= 45):
            print('Hard Problem, ' + str(i) + ' grids to fill')
            for row in prob_state:
                print(*row)
        break

Medium Problem, 41 grids to fill
4 0 0 2 0 1 0 3 0
5 0 1 0 3 6 0 4 7
0 0 6 0 4 0 0 0 1
7 5 0 3 1 0 4 6 9
6 4 0 0 7 0 3 1 8
1 3 0 0 0 9 5 0 2
9 0 0 1 2 0 6 8 0
2 1 0 0 8 0 0 9 5
0 6 0 7 0 0 0 0 0
